<font size=4 face='黑体'>tensorboard网络结构

In [1]:
import tensorflow as tf
import numpy as np

H:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(r'F:\data\mnist_data\MNIST_data',one_hot=True)

Extracting F:\data\mnist_data\MNIST_data\train-images-idx3-ubyte.gz
Extracting F:\data\mnist_data\MNIST_data\train-labels-idx1-ubyte.gz
Extracting F:\data\mnist_data\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting F:\data\mnist_data\MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

#命名空间
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
    
with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.random_normal([784,10])*0.1,name='W')
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
    with tf.name_scope('a'):
        a = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(a)


#交叉熵损失函数
#cross_entropy = -tf.reduce_mean(y*tf.log(tf.clip_by_value(prediction,1e-10,1.0)))
#tf.clip_by_value(v,a,b)表示把v限制在a~b的范围内
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=a))
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(cross_entropy)

init = tf.global_variables_initializer()

#求准确率
with tf.name_scope('accuracy'):
    with tf.name_scope('correct'):
        correct = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))  #equal：判断是否相等并返回bool型数组
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct,tf.float32)) 

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter(r'F:\data\mnist_data\logs',sess.graph)
    for epoch in range(1):
        for batch in range(n_batch):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_x,y:batch_y})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('Iter'+str(epoch)+'Testing  Accuracy:'+str(acc))

Iter0Testing  Accuracy:0.9084


<font size=4 face='黑体'>Tensorboard网络运行

In [3]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)#直方图
        
#命名空间
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
    
with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.random_normal([784,10])*0.1,name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('a'):
        a = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(a)


#交叉熵损失函数
#cross_entropy = -tf.reduce_mean(y*tf.log(tf.clip_by_value(prediction,1e-10,1.0)))
#tf.clip_by_value(v,a,b)表示把v限制在a~b的范围内
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=a))
    tf.summary.scalar('loss',cross_entropy)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(cross_entropy)

init = tf.global_variables_initializer()

#求准确率
with tf.name_scope('accuracy'):
    with tf.name_scope('correct'):
        correct = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))  #equal：判断是否相等并返回bool型数组
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct,tf.float32)) 
        tf.summary.scalar('accuracy',accuracy)
        
#合并所有的summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter(r'F:\data\mnist_data\logs',sess.graph)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            summary,_ = sess.run([merged,train_step],feed_dict={x:batch_x,y:batch_y})
        
        writer.add_summary(summary,epoch)
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('Iter'+str(epoch)+'Testing  Accuracy:'+str(acc))

Iter0Testing  Accuracy:0.9073
Iter1Testing  Accuracy:0.9121
Iter2Testing  Accuracy:0.9183
Iter3Testing  Accuracy:0.9195
Iter4Testing  Accuracy:0.9187
Iter5Testing  Accuracy:0.9221
Iter6Testing  Accuracy:0.9211
Iter7Testing  Accuracy:0.9213
Iter8Testing  Accuracy:0.9229
Iter9Testing  Accuracy:0.9231
Iter10Testing  Accuracy:0.9215
Iter11Testing  Accuracy:0.923
Iter12Testing  Accuracy:0.9222
Iter13Testing  Accuracy:0.9232
Iter14Testing  Accuracy:0.9213
Iter15Testing  Accuracy:0.9232
Iter16Testing  Accuracy:0.9229
Iter17Testing  Accuracy:0.9233
Iter18Testing  Accuracy:0.9224
Iter19Testing  Accuracy:0.9238
Iter20Testing  Accuracy:0.9231


<font size=4 face='黑体'>tensorboard可视化

In [ ]:
from tensorflow.contrib.tensorboard.plugins import projector

# 运行次数
max_steps = 1001
# 图片数量
image_num = 2000

Dir = r'C://Users//yy//'

sess = tf.Session()

#载入图片，把一定数量的图片载入内存
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),
                       trainable=False,name='embedding')

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)#直方图
        
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
    
#显示图片的命名空间
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shaped_input,10) #这里放了10张图片

with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.random_normal([784,10])*0.1,name='W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('a'):
        a = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(a)
        
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=a))
    tf.summary.scalar('loss',cross_entropy)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(cross_entropy)

sess.run(tf.global_variables_initializer())

with tf.name_scope('accuracy'):
    with tf.name_scope('correct'):
        correct = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))  #equal：判断是否相等并返回bool型数组
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct,tf.float32)) 
        tf.summary.scalar('accuracy',accuracy)
        
#产生metadata的文件
if tf.gfile.Exists(Dir+'projector//projector//metadata.tsv'):
    tf.gfile.DeleteRecursively(Dir+'projector//projector//metadata.tsv')  #如果文件存在则先删掉
with open(Dir+'projector//projector//metadata.tsv','w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))  #拿到测试集的所有标签
    for i in range(image_num):
        f.write(str(labels[i]) + '\n')  #写入一部分到文件中

merged = tf.summary.merge_all()

#定义一个writer，写入图
projector_writer = tf.summary.FileWriter(Dir+'projector//projector',sess.graph)
#保存网络模型
saver = tf.train.Saver()
#定义配置文件
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = Dir+'projector//projector//metadata.tsv'
embed.sprite.image_path = Dir+'projector//data//mnist_10k_sprite.png'
#切分图片
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)

for i in range(max_steps):
    batch_x,batch_y = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary,_ = sess.run([merged,train_step],feed_dict={x:batch_x,y:batch_y},
                        options=run_options, run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%03d' % i)
    projector_writer.add_summary(summary,i)
    
    if i%100 == 0:
        acc = sess.run(accuracy, feed_dict={x:mnist.test.iamges, y:mnist.test.labels})
        print('Iter'+str(i)+' Testing Accuracy:'+str(acc))
        
saver.save(sess,Dir+'projector//projector//a_model.ckpt',global_step=max_steps)
projector_writer.close()
sess.close()